In [1]:
# https://huggingface.co/lllyasviel/control_v11p_sd15_openpose

예제

In [3]:
import torch
import os
#from huggingface_hub import HfApi
from pathlib import Path
from diffusers.utils import load_image
from PIL import Image
import numpy as np
from controlnet_aux import OpenposeDetector

from diffusers import (
    ControlNetModel,
    StableDiffusionControlNetPipeline,
    UniPCMultistepScheduler,
)

from datetime import datetime
# 기본 설정값 (변수 선언)
output_dir = "./output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 결과 이미지 저장
date_str = datetime.now().strftime("%Y%m%d%H%M%S")
folder_path = os.path.join(output_dir, date_str[:8])
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

checkpoint = "lllyasviel/control_v11p_sd15_openpose"

# image = load_image(
#     "https://huggingface.co/lllyasviel/control_v11p_sd15_openpose/resolve/main/images/input.png"
# )

prompt = "A person sitting in a wheelchair wearing comfortable clothes."
processor = OpenposeDetector.from_pretrained('lllyasviel/ControlNet')


# Input image path
image_path = "../sample_img\sitting_sample\sample03.jpg"
image = Image.open(image_path).convert("RGB")

control_image = processor(image, hand_and_face=True)

control_save_path = os.path.join(folder_path, "pose_" + date_str[-6:] + ".jpg")

control_image.save(control_save_path)

controlnet = ControlNetModel.from_pretrained(checkpoint, torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16
)
pipe.to("cuda")
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

generator = torch.manual_seed(0)
image = pipe(prompt, num_inference_steps=30, generator=generator, image=control_image).images[0]

result_save_path = os.path.join(folder_path, "sitting_" + date_str[-6:] + ".jpg")
image.save(result_save_path)

100%|██████████| 30/30 [00:05<00:00,  5.79it/s]


stable diffusion sdxl

In [1]:
import os
import torch
from diffusers import StableDiffusionXLPipeline

# Stable Diffusion XL 모델 로드 (fp16을 사용하면 VRAM 절약 가능)
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16, 
    use_safetensors=True
).to("cuda")  # GPU 사용


c:\Users\010\anaconda3\envs\catvton\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading pipeline components...: 100%|██████████| 7/7 [00:06<00:00,  1.07it/s]


In [3]:
from datetime import datetime
# 기본 설정값 (변수 선언)
output_dir = "./output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 결과 이미지 저장
date_str = datetime.now().strftime("%Y%m%d%H%M%S")
folder_path = os.path.join(output_dir, date_str[:8])
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

In [ ]:
# 텍스트 프롬프트 설정
prompt_1 = "A single person sitting in a wheelchair wearing a fashionable outfit, front-facing, no other people, centered composition, clean white background, high-resolution, ultra-realistic, studio lighting, detailed fabric texture"
prompt_2 = "A single person sitting in a wheelchair wearing a casual t-shirt and jeans, front-facing, no other people, centered composition, clean white background, high-resolution, ultra-realistic, studio lighting, detailed fabric texture"
num_images = 200
for i in range(num_images):
    if i <100:
        prompt = prompt_1
    else:
        prompt = prompt_2
    image = pipe(prompt).images[0]
    result_save_path = os.path.join(folder_path, f"wheelchair_person_{i+1:03d}.jpg")
    image.save(result_save_path)

print(f"{num_images} images saved in {folder_path}")



# # 이미지 생성
# image = pipe(prompt).images[0]

# # 이미지 저장 및 표시
# result_save_path = os.path.join(folder_path, "wheelchair_person_" + date_str[-6:] + ".jpg")
# image.save(result_save_path)
# image.show()



100%|██████████| 50/50 [00:16<00:00,  3.03it/s]


200 images saved in ./output\20250216


In [11]:
negative_prompt = "multiple people, crowd, extra arms, extra legs, bad anatomy"
new_negative_prompt = "multiple people, crowd, low quality, blurry, distorted, deformed, watermark, text artifacts, cropped, low resolution, pixelated, unrealistic, extra limbs, missing limbs, disfigured face, bad anatomy, wrong proportions, out of frame, duplicate, poorly drawn hands, poorly drawn feet, unnatural pose, wrong perspective, overexposed, underexposed, monochrome, grayscale, nsfw, low contrast, jpeg artifacts, wrong clothing, mismatched outfit, unrealistic texture, missing wheelchair, incorrect wheelchair shape"

# 텍스트 프롬프트 설정
prompt_1 = "A single person sitting in a wheelchair wearing a fashionable outfit, front-facing, no other people, centered composition, clean white background, high-resolution, ultra-realistic, studio lighting, detailed fabric texture"
prompt_2 = "A single person sitting in a wheelchair wearing a casual t-shirt and jeans, front-facing, no other people, centered composition, clean white background, high-resolution, ultra-realistic, studio lighting, detailed fabric texture"
num_images = 50
for i in range(num_images):
    if i <25:
        prompt = prompt_1
    else:
        prompt = prompt_2
    image = pipe(prompt=prompt, negative_prompt=new_negative_prompt).images[0]
    result_save_path = os.path.join(folder_path, f"wheelchair_person_{i+251:03d}.jpg")
    image.save(result_save_path)

print(f"{num_images} images saved in {folder_path}")

100%|██████████| 50/50 [00:16<00:00,  3.02it/s]


50 images saved in ./output\20250216
